In [4]:
from transformers import pipeline

# Individual tokens inside the sentences are classified into different types.
default_model = "dbmdz/bert-large-cased-finetuned-conll03-english"  # 1.4GB model
# classifier = pipeline("ner")

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [5]:
ner_tokenizer = AutoTokenizer.from_pretrained(default_model,
                                              resume_download=True)

O means the word doesn’t correspond to any entity.

B-PER/I-PER means the word corresponds to the beginning of/is inside a person entity.

B-ORG/I-ORG means the word corresponds to the beginning of/is inside an organization entity.

B-LOC/I-LOC means the word corresponds to the beginning of/is inside a location entity.

B-MISC/I-MISC means the word corresponds to the beginning of/is inside a miscellaneous entity.

O, Outside of a named entity

B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity

I-MIS, Miscellaneous entity

B-PER, Beginning of a person’s name right after another person’s name

I-PER, Person’s name

B-ORG, Beginning of an organisation right after another organisation

I-ORG, Organisation

B-LOC, Beginning of a location right after another location

I-LOC, Location

In [ ]:
ner_model = AutoModelForTokenClassification.from_pretrained(default_model,
                                                            resume_download=True)

In [9]:
label_list = [
"O",       # Outside of a named entity
"B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
"I-MISC",  # Miscellaneous entity
"B-PER",   # Beginning of a person's name right after another person's name
"I-PER",   # Person's name
"B-ORG",   # Beginning of an organisation right after another organisation
"I-ORG",   # Organisation
"B-LOC",   # Beginning of a location right after another location
"I-LOC"    # Location 
]

In [11]:
sentence = "I am from india and I like playing with data"

In [ ]:
sentence_tokenized = ner_tokenizer.tokenize(sentence)

In [ ]:
sentence_input = ner_tokenizer.encode(sentence, return_tensors='pt')

In [ ]:
outputs = ner_model(sentence_input)[0]

In [ ]:
import torch
predictions = torch.argmax(outputs, dim=2) 

In [ ]:
class_sentence = ner_model(sentence)

In [7]:
pos_classifier_model = "vblagoje/bert-english-uncased-finetuned-pos"  # 470 MB model

In [8]:
# Parts of Speech tagging is a Token classification sub-variant

pos_classifier = pipeline("token-classification",
                          model=pos_classifier_model) 

In [12]:
pos_classifier(sentence)

[{'entity': 'PRON',
  'score': 0.99954295,
  'index': 1,
  'word': 'i',
  'start': 0,
  'end': 1},
 {'entity': 'AUX',
  'score': 0.9976078,
  'index': 2,
  'word': 'am',
  'start': 2,
  'end': 4},
 {'entity': 'ADP',
  'score': 0.99935,
  'index': 3,
  'word': 'from',
  'start': 5,
  'end': 9},
 {'entity': 'PROPN',
  'score': 0.99864393,
  'index': 4,
  'word': 'india',
  'start': 10,
  'end': 15},
 {'entity': 'CCONJ',
  'score': 0.9992494,
  'index': 5,
  'word': 'and',
  'start': 16,
  'end': 19},
 {'entity': 'PRON',
  'score': 0.99945444,
  'index': 6,
  'word': 'i',
  'start': 20,
  'end': 21},
 {'entity': 'VERB',
  'score': 0.99256027,
  'index': 7,
  'word': 'like',
  'start': 22,
  'end': 26},
 {'entity': 'VERB',
  'score': 0.99910283,
  'index': 8,
  'word': 'playing',
  'start': 27,
  'end': 34},
 {'entity': 'ADP',
  'score': 0.99935657,
  'index': 9,
  'word': 'with',
  'start': 35,
  'end': 39},
 {'entity': 'NOUN',
  'score': 0.9962528,
  'index': 10,
  'word': 'data',
  'sta